# Mapping components from the iBuildGreen database to LCAbyg constructions and products

This notebook shows how components in the iBuildGreen database (i.e. types of roofs, external walls, etc) are linked with constructions and products from LCAbyg. LCAbyg is a Danish software tool for building LCA, freely accessible at https://lcabyg.dk/en/ <br>
Using LCAbyg is useful because it provides a user-friendly interface to create and modify components, as well as an already existing database of constructions and products. Furthermore, ensuring compatibility with LCAbyg is a safe choice: it will allow us to link the two programs more closely in the future, to use LCAbyg to perform environmental calculations in iBuildGreen, to also import data on "stages" from LCAbyg if needed, etc.

In LCAbyg, buildings are defined as sets of "elements". Each element is made of "constructions", and each construction is made of "products". Each product is then described as a set of LCA "stages". In this notebook, we will only be importing "construction" and "products" for now. We will link each iBuildGreen macrocomponent with one or several constructions, and each construction with one or several products.

This notebook requires access to JSON files containing all constructions and products used in LCAbyg. If these files are not readily available, they can be created manually via LCAbyg, although this is time-consuming. This requires creating a new project in LCAbyg, adding manually to that project all possible constructions and products, and finally exporting the project as a JSON folder. In this case, it might be better to only focus on the constructions and products that are directly needed in the iBuildGreen database.

# Setup

In [ ]:
import json
import psycopg as pg
with open('database_parameters.txt','r') as f: # Text file containing parameters to connect to the database
    params=f.read()
    f.close()

The following generic function is used to compute SQL code in Python and interact with the PostgresQL database:

In [ ]:
def run_sql (DBparameters,SQLcode):
    try:
        # connect to the PostgreSQL database
        connector = pg.connect(DBparameters)

        # create a new cursor
        cur = connector.cursor()

        # execute the SQL statement
        cur.execute(SQLcode)

        # commit the changes to the database
        connector.commit()

        # close communication with the database
        cur.close()

    except (Exception, pg.DatabaseError) as error:
        print(error)

    finally:
        if connector is not None:
            connector.close()

# Constructions

This section shows how to import "constructions" from LCAbyg into the database. First, we need to read the json file containing data on these constructions.

In [ ]:
# Write here the location of the LCAbyg "constructions" json file on your computer
with open('C:/.../LCAbyg/constructions.json') as json_data: # Write the location of the LCAbyg "constructions" file here
    const = json.load(json_data) # Load LCAbyg constructions

In [ ]:
for c in const: # For information - display the names of all constructions
    for key in c.keys():        
        if key=='Node':
            name=c[key]['Construction']['name']['English']
            print(name)        

In iBuildGreen database, we leave open the possibility that each macrocomponent (e.g. external wall type) could be linked with one or several LCAbyg constructions. So we insert LCAbyg constructions into a separate table, called "subcomponents". Later, we will link macrocomponents with the subcomponents table.

First, we define a function to insert items in the subcomponents table.

In [ ]:
def insert_subcomponent(list_of_rows):
    sql = "INSERT INTO subcomponents(lcabyg_id, name, unit, layer, comment) VALUES(%s, %s, %s, %s, %s) ON CONFLICT ON CONSTRAINT subcomponents_pkey DO UPDATE SET (lcabyg_id, name, unit, layer, comment) = (EXCLUDED.lcabyg_id, EXCLUDED.name, EXCLUDED.unit, EXCLUDED.layer, EXCLUDED.comment);"
    connector = None
    bbrid = None
    try:
        # connect to the PostgreSQL database
        connector = pg.connect(params)
        # create a new cursor
        cur = connector.cursor()
        # execute the INSERT statement
        cur.executemany(sql, list_of_rows)
        # commit the changes to the database
        connector.commit()
        # close communication with the database
        cur.close()
    except (Exception, pg.DatabaseError) as error:
        print(error)
    finally:
        if connector is not None:
            connector.close()

Then, we retrieve our subcomponents from the imported json object, as a list:

In [ ]:
list_of_const=[]

for c in const:
    for key in c.keys():        
        if key=='Node':
            ID=c[key]['Construction']['id']
            name=c[key]['Construction']['name']['English']
            unit=c[key]['Construction']['unit']
            layer=c[key]['Construction']['layer']
            comment=c[key]['Construction']['comment']['Danish']
            list_of_const.append((ID,name,unit,layer,comment))

Finally, we insert this list of rows into the database using the function we just defined:

In [ ]:
insert_subcomponent(list_of_const)

# Products

As above, we start by importing the json file containing data on LCAbyg products:

In [ ]:
with open('C:/.../LCAbyg/products.json') as json_data: # Write the location of the LCAbyg "products" file here
    prod = json.load(json_data) 

In [ ]:
for c in prod: # For information - display the names of all products
    for k,p in c.items():        
         print(k,p)

Then we define a function to insert products into the database:

In [ ]:
def insert_product(list_of_prods):
    sql = "INSERT INTO products(lcabyg_id, name, comment) VALUES(%s, %s, %s) ON CONFLICT ON CONSTRAINT products_pkey DO UPDATE SET (lcabyg_id, name, comment) = (EXCLUDED.lcabyg_id, EXCLUDED.name, EXCLUDED.comment);"
    connector = None
    bbrid = None
    try:
        # connect to the PostgreSQL database
        connector = pg.connect(params)
        # create a new cursor
        cur = connector.cursor()
        # execute the INSERT statement
        cur.executemany(sql, list_of_prods)
        # commit the changes to the database
        connector.commit()
        # close communication with the database
        cur.close()
    except (Exception, pg.DatabaseError) as error:
        print(error)
    finally:
        if connector is not None:
            connector.close()

Finally we retrieve information about products from the json object into a list, and we insert this list using the function above.

In [ ]:
list_of_prods=[]
for p in prod:
    for key in p.keys():        
        if key=='Node':
            ID=p[key]['Product']['id']
            name=p[key]['Product']['name']['English']
            comment=p[key]['Product']['comment']
            list_of_prods.append((ID,name,comment))

In [ ]:
insert_product(list_of_prods)

# Mapping table

We have just imported constructions and products into the database, but for now we haven't imported any information on how the two are linked together: we don't know which products are in which constructions. We will now solve this by filling a mapping table "subcomponents_to_products". Each row of the mapping table will indicate the id of a construction, the id of a product, and the amount of this particular product found in this particular construction.

We start by clearing the mapping table if it wasn't already empty:

In [ ]:
run_sql(params,"DELETE FROM subcomponents_to_products")

Links between constructions and products are recorded in the LCAbyg "constructions" json file. They are objects of type "Edge". We can start by retrieving these edges from the "constructions" json file:

In [ ]:
list_of_edges=[]
for c in const:
    for key in c.keys():        
        if key=='Edge':
            ID=c[key][0]['ConstructionToProduct']['id']
            amount=c[key][0]['ConstructionToProduct']['amount']
            unit=c[key][0]['ConstructionToProduct']['unit']
            lifespan=c[key][0]['ConstructionToProduct']['lifespan']
            const_id=c[key][1]
            prod_id=c[key][2]
            list_of_edges.append((ID,const_id,prod_id,amount,unit,lifespan))

Then, as above, we define a function to insert elements into the mapping table:

In [ ]:
def map_component_product(list_of_edges):
    sql = "INSERT INTO subcomponents_to_products(id, subcomponent_id, product_id, amount, unit, lifespan) VALUES(%s, %s, %s, %s, %s, %s) ON CONFLICT ON CONSTRAINT subcomponents_to_products_pkey DO UPDATE SET (id, subcomponent_id, product_id, amount, unit, lifespan) = (EXCLUDED.id, EXCLUDED.subcomponent_id, EXCLUDED.product_id, EXCLUDED.amount, EXCLUDED.unit, EXCLUDED.lifespan);"
    try:
        # connect to the PostgreSQL database
        connector = pg.connect(params)
        
        # fetch construction ids
        cur_const = connector.cursor()
        cur_const.execute('SELECT lcabyg_id FROM subcomponents')
        const_ids=[]
        for i in cur_const.fetchall():
            const_ids.append(i[0])
            
        # fetch product ids       
        cur_prod = connector.cursor()
        cur_prod.execute('SELECT lcabyg_id FROM products')
        prod_ids=[]
        for i in cur_prod.fetchall():
            prod_ids.append(i[0])
        
        # checking for foreign key constraints
        # The LCAbyg edges may include references to products or constructions that are not recorded in the respective files,
        # for instance because a product has been deleted but the corresponding edge hasn't. We need to deal with these cases separately,
        # otherwise the insertion query will raise foreign key constraints (we are trying to insert a row referring to products that do not exist)
        
        valid_edges=[]
        missing_subcomponents=[]
        missing_products=[]
        for edge in list_of_edges:
            if edge[1] in const_ids and edge[2] in prod_ids:
                valid_edges.append(edge)
            if edge[1] not in const_ids:
                missing_subcomponents.append((edge[1],))
            if edge[2] not in prod_ids:
                missing_products.append((edge[2],))
                      
        # insert missing subcomponents
        cur = connector.cursor()
        sql_const = "INSERT INTO subcomponents(lcabyg_id, name) VALUES(%s, 'missing subcomponent') ON CONFLICT ON CONSTRAINT subcomponents_pkey DO NOTHING"
        cur.executemany(sql_const, missing_subcomponents)
        
        # insert missing products
        sql_prod = "INSERT INTO products(lcabyg_id, name) VALUES(%s, 'missing product') ON CONFLICT ON CONSTRAINT products_pkey DO NOTHING"
        cur.executemany(sql_prod, missing_products)
        
        # insert edges in mapping table
        cur.executemany(sql, list_of_edges)
        
        # commit the changes to the database
        connector.commit()
        # close communication with the database
        cur.close()
        cur_const.close()
        cur_prod.close()
    except (Exception, pg.DatabaseError) as error:
        print(error)
    finally:
        if connector is not None:
            connector.close()

Finally we insert the list of edges we just retrieved:

In [ ]:
map_component_product(list_of_edges)

# Handmade components

Because LCAbyg did not include appropriate constructions and products to describe all iBuildGreen macrocomponents, additional constructions and products were added manually. To preserve compatibility with LCAbyg, the constructions and products were created in LCAbyg, exported into a json file, and then imported into the database as previously:

In [ ]:
with open('C:/Users/KJ35FA/Documents/ibuildgreen_data.json') as json_data: # Write the location of the "ibuildgreen_data" file here
    byhand = json.load(json_data) 

list_prods_byhand=[]
for p in byhand:
    for key in p.keys():        
        if key=='Node':
            if 'Product' in p[key].keys():
                ID=p[key]['Product']['id']
                name=p[key]['Product']['name']['English']
                comment=p[key]['Product']['comment']['English']
                list_prods_byhand.append((ID,name,comment)) 

    
list_comp_byhand=[]
for c in byhand:
    for key in c.keys():        
        if key=='Node':
            if 'Construction' in c[key].keys():
                ID=c[key]['Construction']['id']
                name=c[key]['Construction']['name']['English']
                unit=c[key]['Construction']['unit']
                comment=c[key]['Construction']['comment']['Danish']
                if 'layer' in c[key]['Construction'].keys():
                    layer=c[key]['Construction']['layer']
                else:
                    layer=None
                list_comp_byhand.append((ID,name,unit,layer,comment))
    
list_edges_byhand=[]
for c in byhand:
    for key in c.keys():        
        if key=='Edge':
            if 'ConstructionToProduct' in c[key][0].keys():
                ID=c[key][0]['ConstructionToProduct']['id']
                amount=c[key][0]['ConstructionToProduct']['amount']
                unit=c[key][0]['ConstructionToProduct']['unit']
                lifespan=c[key][0]['ConstructionToProduct']['lifespan']
                const_id=c[key][1]
                prod_id=c[key][2]
                list_edges_byhand.append((ID,const_id,prod_id,amount,unit,lifespan))


In [ ]:
insert_product(list_prods_byhand)
insert_subcomponent(list_comp_byhand)

In [ ]:
map_component_product(list_edges_byhand)

# Mapping iBuildGreen macrocomponents to subcomponents/constructions

The following code establishes the link between iBuildGreen macrocomponents and LCAbyg subcomponents in the database. This has to be established manually.

In [ ]:
SQLlink= """
BEGIN;

INSERT INTO public.ext_walls_to_subcomponents(id,ext_wall_id,subcomponent_id,proportion)
VALUES 
(1, 1, 'bdc45ada-2d86-41d4-b62f-6a9aeebd8013',0.8), 
(2, 2, 'ce1e5b61-ccb4-4a82-80ed-3063809ebb72',0.8),
(3, 3, '9c613681-ca0f-4f23-9fd6-d81972922f6c',0.8),
(4, 4, 'b81294fb-a3fa-45e3-b11f-7fc3f4d3cf57',0.8),
(5, 5, '91617bec-ea5d-476e-9ef8-0a156a364928',0.8),
(6, 6, '87e6bf1c-2664-4856-895d-86786809f162',0.8),
(7, 7, '8300127d-69ab-4458-a9b9-9cb248b4445d',0.8),
(8, 8, '7c1e34c6-d00a-428d-8076-c48a7a8804f3',0.8),
(9, 9, '6e9ec424-1bab-4282-b9ff-fd1892122329',0.8),
(10, 10, 'ef3b5d0c-0a72-4b44-aff4-36617526e857',0.8),
(11, 11, '67f9a6b3-7d31-49b6-95f1-f325ec2c8066',0.8),
(12, 12, '1e333afc-137f-4e79-b556-555cd4193248',0.8)
ON CONFLICT ON CONSTRAINT extwalls_to_subcomponents_pkey DO UPDATE SET (id,ext_wall_id,subcomponent_id,proportion)=(EXCLUDED.id,EXCLUDED.ext_wall_id,EXCLUDED.subcomponent_id,EXCLUDED.proportion);

INSERT INTO public.floors_to_subcomponents(id,floor_id,subcomponent_id,proportion)
VALUES 
(1, 1, '8d131da5-ff16-4686-80e7-91269b9d9956',1), 
(2, 2, '348bc8ea-68ce-400b-a5fc-c990462520c1',1),
(3, 3, 'e46431c2-fd9d-4c86-acdc-20d9e51eda3d',1),
(4, 4, '0d45f990-057e-4d42-b9ec-5be5818ecdb6',1),
(5, 5, 'cb9d4f82-d1b3-4ee8-a27a-8aab8029b8f0',1),
(6, 6, '9e551209-32fe-403d-b70f-1cbc08a21194',1),
(7, 7, '07e1418b-c454-4933-8819-215ec288c7d0',1)
ON CONFLICT ON CONSTRAINT floors_to_subcomponents_pkey DO UPDATE SET (id,floor_id,subcomponent_id,proportion)=(EXCLUDED.id,EXCLUDED.floor_id,EXCLUDED.subcomponent_id,EXCLUDED.proportion);

INSERT INTO public.foundations_to_subcomponents(id,foundation_id,subcomponent_id,proportion)
VALUES 
(1, 1, '19fda529-d0b2-424b-b62c-7f68ea62c60e',1), 
(2, 2, 'dc9ec215-7244-4c71-9024-df423dfdfc8c',1),
(3, 3, '5984f57b-21b6-4de8-b815-258c0699ab56',1),
(4, 4, '05af6821-459b-48ca-b00b-0b5d345b88f3',1)
ON CONFLICT ON CONSTRAINT foundations_to_subcomponents_pkey DO UPDATE SET (id,foundation_id,subcomponent_id,proportion)=(EXCLUDED.id,EXCLUDED.foundation_id,EXCLUDED.subcomponent_id,EXCLUDED.proportion);

INSERT INTO public.int_walls_to_subcomponents(id,int_wall_id,subcomponent_id,proportion)
VALUES 
(1, 1, 'bfcee293-03b7-4bc5-ad56-cd2dc8a5c486',1), 
(2, 2, '5de78da5-e80c-4240-aa17-af8ac88db0de',1),
(3, 3, '669ac0df-2b26-4e38-8734-c1e3bf681db2',1),
(4, 4, '0987d73f-8770-4e81-ac8d-ec68836682e7',1),
(5, 5, 'ae720b3d-d6b5-4819-b661-754635f60add',1),
(6, 6, 'c6691ead-ee1b-4863-b1a9-419b7209d9e3',1),
(7, 7, '887b2b33-8bb2-4136-9206-e9e577afadd9',1),
(8, 8, 'b59dc6ff-7849-4ef4-9eb7-9176ea2dd640',1),
(9, 9, 'a9706d09-cff3-41d1-a0a5-9564c9f6b59c',1),
(10, 10, '4b39157e-d8c5-4216-b7c7-da8915dc6d3a',1)
ON CONFLICT ON CONSTRAINT intwalls_to_subcomponents_pkey DO UPDATE SET (id,int_wall_id,subcomponent_id,proportion)=(EXCLUDED.id,EXCLUDED.int_wall_id,EXCLUDED.subcomponent_id,EXCLUDED.proportion);

INSERT INTO public.roof_structures_to_subcomponents(id,roof_structure_id,subcomponent_id,proportion)
VALUES 
(1, 1, '0a5c03db-3826-4013-bcc0-53b877bfeaa0',1), 
(2, 2, '9d5ecb0c-bc0f-402c-a0dc-a5bd0da84e23',1),
(3, 3, '23d6279e-bc45-46e4-8e1f-171329cfe488',1),
(4, 4, 'd1705472-0843-42c2-ae7c-53c168946be6',1),
(5, 5, 'ac175afd-edad-458b-b5e8-ccd2439920c6',1),
(6, 6, 'aa751390-4099-4cc9-81b7-db2e6d0c04b7',1),
(7, 7, '4d59e5b7-b0bb-44cf-9e6d-7060bf4be0aa',1),
(8, 8, '88d5c8b6-59f8-467c-b8cb-c46ea1048844',1),
(9, 9, '3776a0b4-8cb0-4ff3-9304-cc135f138bf3',1)
ON CONFLICT ON CONSTRAINT roofstructures_to_subcomponents_pkey DO UPDATE SET (id,roof_structure_id,subcomponent_id,proportion)=(EXCLUDED.id,EXCLUDED.roof_structure_id,EXCLUDED.subcomponent_id,EXCLUDED.proportion);

INSERT INTO public.roof_covers_to_subcomponents(id,roof_cover_id,subcomponent_id,proportion)
VALUES 
(1, 1, 'b8a24c62-b0dd-43f1-898e-c51857d1acea',1), 
(2, 2, '5e5a3976-ce56-4e5d-8c81-aaacf355775c',1),
(3, 3, 'f6c5b728-dd32-480a-9b02-fa6a6adb6b3c',1),
(4, 4, '68be6a39-5907-4e33-8dc9-29291462cbcc',1),
(5, 5, 'b965cf07-25df-43ce-9f9e-3abf2c8057d4',1),
(6, 6, '2258bb9b-2378-43e4-843f-2c1e4f12e905',1),
(7, 7, '68be6a39-5907-4e33-8dc9-29291462cbcc',1),
(8, 8, 'ccf07cdb-9f83-49ef-a324-11c6c9a9ad57',1),
(9, 9, 'de662e26-9008-41f7-817d-9db0116bedf5',1),
(10, 10, '76250b64-0f59-4ef8-ab3f-2427f9e43c69',1),
(11, 11, '9366b106-548a-42cf-af83-d7842acd43c7',1)
ON CONFLICT ON CONSTRAINT roofcovers_to_subcomponents_pkey DO UPDATE SET (id,roof_cover_id,subcomponent_id,proportion)=(EXCLUDED.id,EXCLUDED.roof_cover_id,EXCLUDED.subcomponent_id,EXCLUDED.proportion);

INSERT INTO public.ground_slabs_to_subcomponents(id,ground_slab_id,subcomponent_id,proportion)
VALUES 
(1, 1, '22e1693f-f4ea-498c-a4a1-e96600457e32',1)
ON CONFLICT ON CONSTRAINT groundslabs_to_subcomponents_pkey DO UPDATE SET (id,ground_slab_id,subcomponent_id,proportion)=(EXCLUDED.id,EXCLUDED.ground_slab_id,EXCLUDED.subcomponent_id,EXCLUDED.proportion);

END;"""

In [ ]:
run_sql(params,SQLlink)

# Adding densities and material types for products

We need to add additional information on material densities and broad categories of material types for relevant products, in order to convert material amounts between volumes and weights and categorize results in a small number of material categories.

In [ ]:
SQLmaterial="""
UPDATE products SET density = 7800, material_type='Metal' WHERE name LIKE '%eel beams%';
UPDATE products SET density = 7800, material_type='Metal' WHERE name LIKE '%eel bars%';
UPDATE products SET density = 7800, material_type='Metal' WHERE name LIKE '%forcement stee%';
UPDATE products SET density = 7800, material_type='Metal' WHERE name LIKE '%eel screws%';
UPDATE products SET density = 7140, material_type='Metal' WHERE name LIKE '%Zin%';
UPDATE products SET density = 2700, material_type='Metal' WHERE name LIKE '%Alumin%';
UPDATE products SET density = 1650, material_type='Concrete' WHERE name LIKE '%ightweight conc%';
UPDATE products SET density = 2400, material_type='Concrete' WHERE name LIKE '%mixed conc%';
UPDATE products SET density = 1650, material_type='Concrete' WHERE name LIKE '%of tiles, con%';
UPDATE products SET density = 650, material_type='Wood' WHERE name LIKE 'Timber%';
UPDATE products SET density = 650, material_type='Wood' WHERE name LIKE 'Wood beam%';
UPDATE products SET density = 650, material_type='Wood' WHERE name LIKE 'Plywood%';
UPDATE products SET density = 650, material_type='Wood' WHERE name LIKE '%spruce%';
UPDATE products SET density = 50, material_type='Wool' WHERE name LIKE '%wool%';
UPDATE products SET density = 50, material_type='Other' WHERE name LIKE '%amp insulat%';
UPDATE products SET density = 100, material_type='Other' WHERE name LIKE '%EPDM%';
UPDATE products SET density = 100, material_type='Other' WHERE name LIKE '%Foil%';
UPDATE products SET density = 100, material_type='Other' WHERE name LIKE '%PE%';
UPDATE products SET density = 2580, material_type='Glass' WHERE name LIKE '%glaz%';
UPDATE products SET density = 50, material_type='Other' WHERE name LIKE '%itumen sheet%';
UPDATE products SET density = 650, material_type='Wood' WHERE name LIKE '%pine and spruce (skeleton)%';
UPDATE products SET density = 2000, material_type='Cement_mortar' WHERE name LIKE '%ement mortar%';
UPDATE products SET density = 2000, material_type='Cement_mortar' WHERE name LIKE '%ime mortar%';
UPDATE products SET density = 2000, material_type='Cement_mortar' WHERE name LIKE '%ement screed%';
UPDATE products SET density = 700, material_type='Gypsum_plaster' WHERE name LIKE '%ime plaster%';
UPDATE products SET density = 700, material_type='Gypsum_plaster' WHERE name LIKE '%psum plaster%';
UPDATE products SET density = 700, material_type='Gypsum_plaster' WHERE name LIKE '%terior plaster%';
UPDATE products SET density = 1600, material_type='Other' WHERE name LIKE '%slate%';
UPDATE products SET density = 2000, material_type='Aggregates' WHERE name LIKE '%rushed stone%';
UPDATE products SET density = 1600, material_type='Clay' WHERE name IN ('Facing brick', 'Foundation brick or stones', 'Foundation brick', 'Hollow core brick', 'Internal brick', 'Roof tile');
UPDATE products SET density = 650, material_type='Concrete' WHERE name LIKE '%erated%';
UPDATE products SET density = 1650, material_type='Cement_mortar' WHERE name LIKE '%ibre cement%';
UPDATE products SET density = 1000, material_type='Gypsum_plaster' WHERE name LIKE '%alcium silicate boar%';
UPDATE products SET density = 1000, material_type='Gypsum_plaster' WHERE name LIKE '%psum boar%';
UPDATE products SET density = 1000, material_type='Gypsum_plaster' WHERE name LIKE '%psum wallboar%';
UPDATE products SET density = 500, material_type='Wood' WHERE name LIKE '%hipboard%';
UPDATE products SET density = 1700, material_type='Clay' WHERE name LIKE 'Clay%';
UPDATE products SET density = 1700, material_type='Other' WHERE name LIKE '%Baustroh%';
"""

In [ ]:
run_sql(params,SQLmaterial)